In [2]:
import pandas as pd
import datetime as DT
import pyspark as ps
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array

In [3]:
from pyspark import SparkContext
sc=SparkContext("local" , "MyApp")

In [4]:
buyclicksDF=pd.read_csv('~/Capstone/GameData/flamingo-data/buy-clicks.csv')
buyclicksDF=buyclicksDF.rename(columns= lambda x: x.strip()) #removing whitespaces from headers
buyClicks=buyclicksDF[['userId','userSessionId','txId','team','price']]

In [5]:
userDF=pd.read_csv('~/Capstone/GameData/flamingo-data/users.csv')
userDF=userDF.rename(columns= lambda x: x.strip()) #removing whitespaces from headers
users=userDF[['userId','dob','country']]
now=pd.Timestamp(DT.datetime.now())
users['dob']=(now - (pd.to_datetime(users['dob']))).astype('<m8[Y]')
users.columns=['userId','age','country']

/home/cloudera/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
combinedUserBuys=buyClicks.merge(users, on='userId')

In [7]:
adclicksDF=pd.read_csv('~/Capstone/GameData/flamingo-data/ad-clicks.csv')
adclicksDF=adclicksDF.rename(columns= lambda x: x.strip()) #removing whitespaces from headers
adClicks=adclicksDF[['userId','userSessionId']]
adClicks['adCount'] = 1
adClicks.head(n=5)

/home/cloudera/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,userId,userSessionId,adCount
0,611,5809,1
1,1874,5705,1
2,2139,5791,1
3,212,5756,1
4,1027,5920,1


In [8]:
adClicksAgg = adClicks.groupby(['userId','userSessionId']).size().reset_index()
adClicksAgg.columns = ['userId','userSessionId','totalAdClicks'] #rename the columns
#adClicksAgg1 = adClicksAgg.groupby(['userId','userSessionId']).max().reset_index()
adClicksAgg.head(n=10)
#adClicksAgg = adClicksAgg.groupby(['userId','userSessionId','adCategory']).max()
#adClicksAgg = adClicksAgg.reset_index()
#adClicksAgg.columns = ['userInfo','totalClicks'] #rename the columns
#adClicksAgg.head(n=10)

,userId,userSessionId,totalAdClicks
0,1,5835,6
1,1,7847,4
2,1,10041,2
3,1,12713,3
4,1,16232,9
5,1,21014,8
6,1,26938,10
7,1,34802,2
8,8,27410,1
9,8,27918,5


(2870,)

In [8]:
combinedBuyCategory = buyClicks.merge(adClicks, on='txId')